In [50]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List
import torch
from pprint import pprint

In [51]:
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.pad_token_id = model.config.eos_token_id

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [64]:
def to_tokens_and_probs(model: AutoModelForCausalLM, tokenizer: AutoTokenizer, input_texts: List[str]):
    """
    This function takes a list of input texts and returns a list of tuples (token, prob) for each token in the input text.
    Reference: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/17    

    Args:
        model (AutoModelForCausalLM): _description_
        tokenizer (AutoTokenizer): _description_
        input_texts (List[str]): _description_

    Returns:
        _type_: _description_
    """
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token, p in zip(input_sentence, input_probs):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item()))
        batch.append(text_sequence)
    return batch

In [65]:
sample_prompt = ["The capital of France is Paris."]
token_probs = to_tokens_and_logprobs(model, tokenizer, sample_prompt)


In [66]:
token_probs

[[('The', 0.08746550977230072),
  (' capital', 8.743480429984629e-05),
  (' of', 0.14376778900623322),
  (' France', 0.0054758754558861256),
  (' is', 0.24484363198280334),
  (' Paris', 0.006779637187719345),
  ('.', 0.3360891044139862)]]